In [ ]:
from tensorflow import keras
from keras.models import load_model

In [ ]:
model = keras.models.load_model('my_model.h5')

In [79]:
user = 1

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from time import sleep
from keras.preprocessing.image import img_to_array
class_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
final_mood=[]
def face_detector(img):
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(img, 1.3, 5)
    if faces is ():
        return (0,0,0,0), np.zeros((48,48), np.uint8), img
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]

    try:
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation = cv2.INTER_AREA)
    except:
        return (x,w,y,h), np.zeros((48,48), np.uint8), img
    return (x,w,y,h), roi_gray, img

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    rect, face, image = face_detector(frame)
    if np.sum([face]) != 0.0:
        roi = face.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)

        # make a prediction on the ROI, then lookup the class
        preds = model.predict(roi)[0]
        label = class_labels[preds.argmax()]  
        #print(label)
        final_mood.append(label)
        label_position = (rect[0] + int((rect[1]/2)), rect[2] + 25)
        cv2.putText(image, label, label_position , cv2.FONT_HERSHEY_SIMPLEX,2, (0,255,0), 3)
    else:
        cv2.putText(image, "No Face Found", (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,2, (0,255,0), 3)
        
    cv2.imshow("emotion detection",image)
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()  

In [ ]:
final_mood

In [ ]:
mood= max(final_mood,key=final_mood.count)

In [ ]:
mood

In [154]:
mood = 'angry'
userid = user

**Build up a hybrid recommender system based on MovieLens database using content-based filtering and collaborative filtering algorithms**

In [155]:
import numpy as np
import pandas as pd 
import os

In [156]:
!pip install fuzzywuzzy

In [157]:
!pip install surprise

In [158]:
#run either
#!pip install surprise
# conda install -c conda-forge scikit-surprise

In [159]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz

from surprise import SVD, SVDpp, KNNBasic
from surprise import Dataset
from surprise.model_selection import cross_validate,train_test_split, GridSearchCV
from surprise import NormalPredictor
from surprise import Reader

import re 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk

**Read and have a look at the data**

The data is downloaded from MovieLens dataset:
https://grouplens.org/datasets/movielens/latest/

The files needed are "movies.csv" and "ratings.csv"

In [160]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aryanringshia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [161]:
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")
genre_df = pd.read_csv("genres.csv")

In [162]:
# g = genre_df.loc[(genre_df['user']==1) & (genre_df['mood']==mood)]['genre'].iloc[0].split(',')[0].replace("'",'').replace("[",'').replace("]",'')

In [163]:
g = genre_df.loc[(genre_df['user']==1) & (genre_df['mood']==mood)]['genre'][0]

In [164]:
g

"['Comedy']"

In [165]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [166]:
g

"['Comedy']"

In [167]:
movies = movies[movies['genres'].str.contains(g)].reset_index()

In [168]:
favs = ratings.loc[(ratings['userId']==1)].sort_values(by=['rating'], ascending=False)[:5]['movieId'].tolist()

In [169]:
ratings.loc[(ratings['userId']==1)].sort_values(by=['rating'], ascending=False)[:5]['movieId'].tolist()

[912, 1136, 4262, 1221, 2959]

In [170]:
str1 = ''
str1.join(movies.loc[(movies['movieId']==1136)]['title'].to_list())

'Monty Python and the Holy Grail (1975)'

In [171]:
favs_list = []
for x in favs:
    if movies.loc[(movies['movieId']==x)]['title'].empty == False:
        favs_list.append(str1.join(movies.loc[(movies['movieId']==1136)]['title'].to_list()))

In [172]:
favs_list = list(set(favs_list))
favs_list

['Monty Python and the Holy Grail (1975)']

In [173]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [174]:
movies.head(10)

,index,movieId,title,genres
0,0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,2,Jumanji (1995),Adventure|Children|Fantasy
2,2,3,Grumpier Old Men (1995),Comedy|Romance
3,3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,4,5,Father of the Bride Part II (1995),Comedy
5,5,6,Heat (1995),Action|Crime|Thriller
6,6,7,Sabrina (1995),Comedy|Romance
7,7,8,Tom and Huck (1995),Adventure|Children
8,8,9,Sudden Death (1995),Action
9,9,10,GoldenEye (1995),Action|Adventure|Thriller


In [175]:
ratings_array = ratings['rating'].unique()
max_rating = np.amax( ratings_array )
min_rating = np.amin( ratings_array )
print( ratings_array )

[4.  1.5 3.  0.5 4.5 2.5 3.5 5.  2.  1. ]


In [176]:
movie_map = pd.Series(movies.movieId.values,index=movies.title).to_dict()
reverse_movie_map = {v: k for k, v in movie_map.items()}
movieId_to_index_map = pd.Series(movies.index.values,index=movies.movieId).to_dict()
movieId_all_array = movies['movieId'].unique()

In [177]:
def get_movieId( movie_name ):
    """
    return the movieId which is corresponding to the movie name

    Parameters
    ----------
    movie_name: string, the name of the movie w/ or w/o the year

    Return
    ------
    the movieId
    """

    # If luckily the movie name is 100% equal to a name writen in the database,
    # then return the id corresponding to the name.
    # Or...we need to consider the similarity between strings 
    if (movie_name in movie_map):
      return movie_map[movie_name]
    else:
      similar = []
      for title, movie_id in movie_map.items():
        ratio = fuzz.ratio(title.lower(), movie_name.lower())
        if ( ratio >= 60):
          similar.append( (title, movie_id, ratio ) )
      if (len(similar) == 0):
        print("Oh! This movie does not exist in the database.")
      else:
        match_item = sorted( similar , key=lambda x: x[2] )[::-1]
        print( "The matched item might be:", match_item[0][0], ", ratio=",match_item[0][2] )
        return match_item[0][1]

**Build up the content-based filtering algorithm with pairwise approach in TF-IDF vector space**

In [178]:
def tokenizer(text):
  torkenized = [PorterStemmer().stem(word).lower() for word in text.split('|') if word not in stopwords.words('english')]
  return torkenized

In [179]:
tfid=TfidfVectorizer(analyzer='word', tokenizer=tokenizer)

In [180]:
tfidf_matrix = tfid.fit_transform(movies['genres'])

In [181]:
cos_sim = cosine_similarity(tfidf_matrix,tfidf_matrix)

In [182]:
print(tfidf_matrix.shape)
print(cos_sim.shape)
print(movies.shape)

(10272, 20)
(10272, 10272)
(10272, 4)


**Build up the Singular Value Decomposition (SVD) matrix factorization model in collaborative filtering algorithm**

In [183]:
features = ['userId','movieId', 'rating']
reader = Reader(rating_scale=(min_rating, max_rating))
data = Dataset.load_from_df(ratings[features], reader)
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

In [184]:
gs.fit(data)

In [185]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])


0.8902151090922787
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [186]:
best_params = gs.best_params['rmse']
model_svd = gs.best_estimator['rmse']
model_svd.fit(data.build_full_trainset())

In [187]:
best_params

{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}

In [188]:
def get_rating_from_prediction( prediction, ratings_array ):
    """
    return the closest rating number to the prediction value

    Parameters
    ----------
    prediction: float, the prediction value from the model

    ratings_array: the 1D array of the discrete rating number

    Return
    ------
    the rating number corresponding to the prediction value
    """
    rating = ratings_array[ np.argmin( [ np.abs(item - prediction) for item in ratings_array ] ) ]
    return rating

In [189]:
prediction = model_svd.predict(1,1)
print("rating", ratings[(ratings.userId ==1 ) & (ratings.movieId ==1 ) ]['rating']  )
print("prediction",prediction.est)

rating Series([], Name: rating, dtype: float64)
prediction 3.7318764546355645


**Make movie recommendation (item-based)**

In [190]:
def make_recommendation_item_based( similarity_matrix ,movieId_all_array, ratings_data, id_to_movie_map, movieId_to_index_map, fav_movie_list, n_recommendations, userId=-99):
    """
    return top n movie recommendation based on user's input list of favorite movies
    Currently, fav_movie_list only support one input favorate movie

    Parameters
    ----------
    similarity_matrix: 2d array, the pairwise similarity matrix

    movieId_all_array: 1d array, the array of all movie Id

    ratings_data: ratings data

    id_to_movie_map: the map from movieId to movie title

    movieId_to_index_map: the map from movieId to the index of the movie dataframe

    fav_movie_list: list, user's list of favorite movies

    n_recommendations: int, top n recommendations

    userId: int optional (default=-99), the user Id
            if userId = -99, the new user will be created
            if userId = -1, the latest inserted user is chosen

    Return
    ------
    list of top n movie recommendations

    """

    if (userId == -99):
      userId = np.amax( ratings_data['userId'].unique() ) + 1
    elif (userId == -1):
      userId = np.amax( ratings_data['userId'].unique() )

    movieId_list = []
    for movie_name in fav_movie_list:
      movieId_list.append( get_movieId(movie_name) )    

    # Get the movie id which corresponding to the movie the user didn't watch before
    movieId_user_exist = list( ratings_data[ ratings_data.userId==userId ]['movieId'].unique() )
    movieId_user_exist = movieId_user_exist + movieId_list
    movieId_input = []
    for movieId in movieId_all_array:
      if (movieId not in movieId_user_exist):
         movieId_input.append( movieId )


    index = movieId_to_index_map[movieId_list[0]]
    cos_sim_scores=list(enumerate(similarity_matrix[index]))
    cos_sim_scores=sorted(cos_sim_scores,key=lambda x:x[1],reverse=True) 
  
    topn_movieIndex = []
    icount = 0
    for i in range(len(cos_sim_scores)):
      if( cos_sim_scores[i][0] in [movieId_to_index_map[ids] for ids in movieId_input ]  ):
        icount += 1
        topn_movieIndex.append( cos_sim_scores[i][0] )
        if( icount == n_recommendations ):
          break
    
    topn_movie = [ movies.loc[index].title for index in topn_movieIndex ]
    return topn_movie



**Make movie recommendation (user-based)**



In [191]:
def make_recommendation_user_based(best_model_params, movieId_all_array, ratings_data, id_to_movie_map,
                        fav_movie_list, n_recommendations, userId=-99 ):
    """
    return top n movie recommendation based on user's input list of favorite movies
    Currently, fav_movie_list only support one input favorate movie


    Parameters
    ----------
    best_model_params: dict, {'iterations': iter, 'rank': rank, 'lambda_': reg}

    movieId_all_array: the array of all movie Id

    ratings_data: ratings data

    id_to_movie_map: the map from movieId to movie title

    fav_movie_list: list, user's list of favorite movies

    n_recommendations: int, top n recommendations

    userId: int optional (default=-99), the user Id
            if userId = -99, the new user will be created
            if userId = -1, the latest inserted user is chosen

    Return
    ------
    list of top n movie recommendations
    """

    movieId_list = []
    for movie_name in fav_movie_list:
      movieId_list.append( get_movieId(movie_name) )

    if (userId == -99):
      userId = np.amax( ratings_data['userId'].unique() ) + 1
    elif (userId == -1):
      userId = np.amax( ratings_data['userId'].unique() )

    ratings_array = ratings['rating'].unique()
    max_rating = np.amax( ratings_array )
    min_rating = np.amin( ratings_array )
    
    # create the new row which corresponding to the input data
    user_rows = [[userId, movieId, max_rating] for movieId in movieId_list]
    df = pd.DataFrame(user_rows, columns =['userId', 'movieId', 'rating']) 
    train_data = pd.concat([ratings_data, df], ignore_index=True, sort=False)

    # Get the movie id which corresponding to the movie the user didn't watch before
    movieId_user_exist = train_data[ train_data.userId==userId ]['movieId'].unique()
    movieId_input = []
    for movieId in movieId_all_array:
      if (movieId not in movieId_user_exist):
         movieId_input.append( movieId )

    reader = Reader(rating_scale=(min_rating, max_rating))

    data = Dataset.load_from_df(train_data, reader)

    model = SVD(**best_model_params)
    model.fit(data.build_full_trainset())

    predictions = []
    for movieId in movieId_input:
      predictions.append( model.predict(userId,movieId) )

    
    sort_index = sorted(range(len(predictions)), key=lambda k: predictions[k].est, reverse=True)
    topn_predictions = [ predictions[i].est for i in sort_index[0:min(n_recommendations,len(predictions))] ]
    topn_movieIds = [ movieId_input[i] for i in sort_index[0:min(n_recommendations,len(predictions))] ]
    topn_rating = [ get_rating_from_prediction( pre, ratings_array ) for pre in topn_predictions ]

    topn_movie = [ id_to_movie_map[ ids ] for ids in topn_movieIds ]
    return topn_movie



**Make a recommendation!**

In [192]:
my_favorite_movies = favs_list
#['Iron Man']

# get recommends
n_recommendations = 20

recommends_item_based = make_recommendation_item_based( 
    similarity_matrix = cos_sim,
    movieId_all_array = movieId_all_array,
    ratings_data = ratings[features], 
    id_to_movie_map = reverse_movie_map, 
    movieId_to_index_map = movieId_to_index_map,
    fav_movie_list = my_favorite_movies, 
    n_recommendations = n_recommendations)

recommends_user_based = make_recommendation_user_based(
    best_model_params = best_params, 
    movieId_all_array = movieId_all_array,
    ratings_data = ratings[features], 
    id_to_movie_map = reverse_movie_map, 
    fav_movie_list = my_favorite_movies, 
    n_recommendations = n_recommendations)

print("-------------Search based on item's content similarity--------------------")
print('The movies similar to' , my_favorite_movies , ':' )
for i, title in enumerate(recommends_item_based):
    print(i+1, title)  
if( len(recommends_item_based) < n_recommendations ):
  print("Sadly, we couldn't offer so many recommendations :(")    

print("--------------Search based on similarity between user's preference--------------------------------------")
print('The users like' , my_favorite_movies , 'also like:')
for i, title in enumerate(recommends_user_based):
    print(i+1, title)
if( len(recommends_user_based) < n_recommendations ):
  print("Sadly, we couldn't offer so many recommendations :(")

-------------Search based on item's content similarity--------------------
The movies similar to ['Monty Python and the Holy Grail (1975)'] :
1 Dogma (1999)
2 Hook (1991)
3 Adventures of Baron Munchausen, The (1988)
4 Erik the Viking (1989)
5 Black Knight (2001)
6 Jabberwocky (1977)
7 10th Kingdom, The (2000)
8 Asterix & Obelix: Mission Cleopatra (Astérix & Obélix: Mission Cléopâtre) (2002)
9 Life Aquatic with Steve Zissou, The (2004)
10 Nim's Island (2008)
11 Colour of Magic, The (Terry Pratchett's The Colour of Magic) (2008)
12 Gulliver's Travels (2010)
13 Dark Crystal, The (1982)
14 Excalibur (1981)
15 Sinbad and the Eye of the Tiger (1977)
16 Lord of the Rings: The Fellowship of the Ring, The (2001)
17 Harry Potter and the Chamber of Secrets (2002)
18 Lord of the Rings: The Two Towers, The (2002)
19 Thief of Bagdad, The (1940)
20 Jungle Book (1942)
--------------Search based on similarity between user's preference--------------------------------------
The users like ['Monty Python 